In [ ]:
import helpFunctions as hf 
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import imread
import pandas as pd

In [ ]:
dirIn = 'data/'

In [ ]:
## Example of loading a multi spectral image
multiIm, annotationIm = hf.loadMulti('multispectral_day01.mat' , 'annotation_day01.png', dirIn)

# multiIm is a multi spectral image - the dimensions can be seen by
display(multiIm.shape)

## annotationIm is a binary image with 3 layers
display(annotationIm.shape)

In [ ]:
## Show image óf spectral band 7
plt.imshow(multiIm[:,:,6])
plt.show()

In [ ]:
# Example with meat- and fat annotation
[fatPix, fatR, fatC] = hf.getPix(multiIm, annotationIm[:,:,1])
[meatPix, meatR, meatC] = hf.getPix(multiIm, annotationIm[:,:,2])

# Plot the mean values for pixels with meat and fat respectively
plt.plot(np.mean(meatPix,0),'b')
plt.plot(np.mean(fatPix,0),'r')
plt.show()

In [ ]:
## The function showHistogram makes a histogram that is returned as a vector

# Here is an example - last argument tells the function to plot the histogram for meat and fat
h = hf.showHistograms(multiIm, annotationIm[:,:,2:3], 2, 1)

## The histogram is also in h
# But not truncated like in the plot. If we wnat to avoid plotting all 256 dimensions, 
# we can do like below, and only plot the first 50 values

plt.plot(h[0:50:,:])
plt.show()

In [ ]:
## The function setImagePix produces a colour image
# where the pixel coordinates are given as input

# Load RGB image
imRGB = imread(dirIn + 'color_day01.png')

# Pixel coordinates for the fat annotation
[fatPix, fatR, fatC] = hf.getPix(multiIm, annotationIm[:,:,1])

# Concatenate the pixel coordinates to a matrix
pixId = np.stack((fatR, fatC), axis=1)

# Make the new images
rgbOut = hf.setImagePix(imRGB, pixId)
plt.imshow(rgbOut)
plt.show()

In [ ]:
# Band 15 histogram
l = 14
plt.hist(meatPix[:, l], bins=50, alpha=0.5, label="meat")
plt.hist(fatPix[:, l],  bins=50, alpha=0.5, label="fat")
plt.legend()
plt.show()

In [ ]:
def train_thr_model(fatPix, meatPix):
    Nf, Nm = fatPix.shape[0], meatPix.shape[0]
    err = np.zeros(19)
    thr = np.zeros(19)

    for l in range(19):
        mu_f = np.mean(fatPix[:, l])
        mu_m = np.mean(meatPix[:, l])
        thr[l] = 0.5 * (mu_f + mu_m)

        fat_wrong  = np.sum(fatPix[:, l]  <  thr[l])
        meat_wrong = np.sum(meatPix[:, l] >= thr[l])
        err[l] = (fat_wrong + meat_wrong) / (Nf + Nm)

    best_l = int(np.argmin(err))
    best_band = best_l + 1
    thr_best = thr[best_l]
    return best_l, best_band, thr_best

def eval_thr_model(fatPix, meatPix, best_l, thr_best):
    Nf, Nm = fatPix.shape[0], meatPix.shape[0]
    fat_wrong  = np.sum(fatPix[:, best_l]  <  thr_best)
    meat_wrong = np.sum(meatPix[:, best_l] >= thr_best)
    return (fat_wrong + meat_wrong) / (Nf + Nm)

best_l, best_band, thr_best = train_thr_model(fatPix, meatPix)
err_best = eval_thr_model(fatPix, meatPix, best_l, thr_best)

best_band, thr_best, err_best


In [ ]:
salami_an = annotationIm[:, :, 0].astype(bool)

cls = np.zeros(multiIm.shape[:2], dtype=np.uint8)
cls[salami_an] = 1
cls[salami_an & (multiIm[:, :, best_l] >= thr_best)] = 2

plt.figure()
plt.imshow(cls)
plt.title(f"Threshold classification ")
plt.axis("off")
plt.show()

In [ ]:
# single-band threshold model proportion
salami_single = cls[salami_an]
n_meat_LDA_prior = len(salami_single[salami_single == 1])
n_fat_LDA_prior = len(salami_single[salami_single == 2])
print(n_fat_LDA_prior / (n_fat_LDA_prior + n_meat_LDA_prior))

In [ ]:
def train_lda(fatPix, meatPix):
    #Number of annotated fat and meat pixels
    Nf = fatPix.shape[0]
    Nm = meatPix.shape[0]

    # Means 
    mu_f = np.mean(fatPix, axis=0)
    mu_m = np.mean(meatPix, axis=0)

    # Covariance matrices for fat and meat
    Sigma_f = np.cov(fatPix, rowvar=False, ddof=1)
    Sigma_m = np.cov(meatPix, rowvar=False, ddof=1)
    # Pooled covariance (LDA assumption)
    Sigma = ((Nf-1)*Sigma_f + (Nm-1)*Sigma_m) / ((Nf-1)+(Nm-1))
    Sigma_inv = np.linalg.pinv(Sigma)

    # rewrite (S_f - S_m) to x^T*w+c
    w = Sigma_inv @ (mu_f - mu_m)
    c = -0.5 * (mu_f @ Sigma_inv @ mu_f - mu_m @ Sigma_inv @ mu_m)
    return Sigma, w, c

def eval_lda(fatPix, meatPix, w, c):
    Nf, Nm = fatPix.shape[0], meatPix.shape[0]
    fat_wrong  = np.sum((fatPix  @ w + c) < 0)
    meat_wrong = np.sum((meatPix @ w + c) >= 0)
    return (fat_wrong + meat_wrong) / (Nf + Nm)

Sigma,w,c = train_lda(fatPix, meatPix)

err_lda = eval_lda(fatPix, meatPix, w, c)

err_lda

In [ ]:
# Extract only salami pixels as vector
X_salami = multiIm[salami_an, :].astype(np.float64)

s_salami = X_salami @ w + c

cls_lda = np.zeros(multiIm.shape[:2], dtype=np.uint8)  
cls_lda[salami_an] = 1                 
cls_lda[salami_an] = np.where(s_salami >= 0, 2, 1)

plt.figure(figsize=(7, 7))
plt.imshow(cls_lda)
plt.title("LDA classification")
plt.axis("off")
plt.show()

In [ ]:
# multispectral LDA proportion
n_meat_LDA_prior = np.sum(cls_lda[salami_an] == 1)
n_fat_LDA_prior = np.sum(cls_lda[salami_an] == 2)
n_fat_LDA_prior / (n_fat_LDA_prior + n_meat_LDA_prior)

In [ ]:
# Plot of correlation matrix
D = np.sqrt(np.diag(Sigma))
Corr = Sigma / np.outer(D, D)

plt.figure(figsize=(6, 5))
plt.imshow(Corr, vmin=-1, vmax=1)
plt.colorbar(label="correlation")
plt.title("Pooled correlation matrix")
plt.xlabel("Band")
plt.ylabel("Band")
plt.show()

In [ ]:
# helper for loading days
def load_day(d):
    ms_file = f"multispectral_day{d:02d}.mat"
    an_file = f"annotation_day{d:02d}.png"
    return hf.loadMulti(ms_file, an_file, dirIn)

In [ ]:
test_days = [6, 13, 20, 28]

results = []

# computes errors of threshold and LDA models trained on day 1 data
for d in test_days:
    multiIm_d, annotationIm_d = load_day(d)

    # extract evaluation pixels
    fatPix_d,  _, _ = hf.getPix(multiIm_d, annotationIm_d[:, :, 1])
    meatPix_d, _, _ = hf.getPix(multiIm_d, annotationIm_d[:, :, 2])

    Nf_d = fatPix_d.shape[0]
    Nm_d = meatPix_d.shape[0]

    # Threshold error
    err_thr_d = eval_thr_model(fatPix_d, meatPix_d, best_l, thr_best)

    # LDA error
    err_lda_d = eval_lda(fatPix_d, meatPix_d,w,c)

    results.append({"day": d, "thr_err": err_thr_d, "lda_err": err_lda_d})


df = pd.DataFrame(results)
print(df.to_string(index=False))

In [ ]:
days = [1, 6, 13, 20, 28]
n = len(days)

results_all_days = []

# trains models for each day and tests on remaining
for i, d_train in enumerate(days):
    #load data
    multi_train, ann_train = load_day(d_train)
    fat_train, _, _  = hf.getPix(multi_train, ann_train[:, :, 1])
    meat_train, _, _ = hf.getPix(multi_train, ann_train[:, :, 2])

    # train both models on this day
    best_l, _, thr_best = train_thr_model(fat_train, meat_train)
    _, w, c = train_lda(fat_train, meat_train)

    # test on other days
    for j, d_test in enumerate(days):
        if d_test == d_train:
            continue

        multi_test, ann_test = load_day(d_test)
        fat_test, _, _  = hf.getPix(multi_test, ann_test[:, :, 1])
        meat_test, _, _ = hf.getPix(multi_test, ann_test[:, :, 2])

        thr_err = eval_thr_model(fat_test, meat_test, best_l, thr_best)
        lda_err = eval_lda(fat_test, meat_test, w, c)

        results_all_days.append({"train": d_train,
                                 "test": d_test,
                                 "thr_err": thr_err,
                                 "lda_err": lda_err})

# Print
df = pd.DataFrame(results_all_days)
print(df.to_string(index=False))


In [ ]:
#Train LDA without prior knowledge
Sigma,w,c = train_lda(fatPix, meatPix)

# Prior knowledge
pf=0.3
pm=0.7
log_prior=np.log(pf/pm)
c_w_prior = c + log_prior

err_lda = eval_lda(fatPix, meatPix, w, c)
err_lda_w_prior = eval_lda(fatPix, meatPix, w, c_w_prior)

err_lda_w_prior, err_lda

In [ ]:
test_days = [6, 13, 20, 28]

results = []

# computes errors of threshold and LDA models trained on day 1 data
for d in test_days:
    multiIm_d, annotationIm_d = load_day(d)

    # extract evaluation pixels
    fatPix_d,  _, _ = hf.getPix(multiIm_d, annotationIm_d[:, :, 1])
    meatPix_d, _, _ = hf.getPix(multiIm_d, annotationIm_d[:, :, 2])

    Nf_d = fatPix_d.shape[0]
    Nm_d = meatPix_d.shape[0]

    # Threshold error
    err_thr_d = eval_thr_model(fatPix_d, meatPix_d, best_l, thr_best)

    # LDA error
    err_lda_d = eval_lda(fatPix_d, meatPix_d,w,c)

    # LDA with priors
    err_lda_w_prior_d = eval_lda(fatPix_d, meatPix_d,w,c_w_prior)

    results.append({"day": d,
                    "thr_err": err_thr_d,
                    "lda_err": err_lda_d,
                    "lda_err_w_prior": err_lda_w_prior_d})


df = pd.DataFrame(results)
print(df.to_string(index=False))

In [ ]:
days = [1, 6, 13, 20, 28]
n = len(days)

results_all_days = []

# trains models for each day and tests on remaining
for i, d_train in enumerate(days):
    #load data
    multi_train, ann_train = load_day(d_train)
    fat_train, _, _  = hf.getPix(multi_train, ann_train[:, :, 1])
    meat_train, _, _ = hf.getPix(multi_train, ann_train[:, :, 2])

    # train both models on this day
    best_l, _, thr_best = train_thr_model(fat_train, meat_train)
    _, w, c = train_lda(fat_train, meat_train)
    c_w_prior_d = c + log_prior

    # test on other days
    for j, d_test in enumerate(days):
        if d_test == d_train:
            continue

        multi_test, ann_test = load_day(d_test)
        fat_test, _, _  = hf.getPix(multi_test, ann_test[:, :, 1])
        meat_test, _, _ = hf.getPix(multi_test, ann_test[:, :, 2])

        thr_err = eval_thr_model(fat_test, meat_test, best_l, thr_best)
        lda_err = eval_lda(fat_test, meat_test, w, c)
        lda_w_prior_err = eval_lda(fat_test, meat_test, w, c_w_prior_d)

        results_all_days.append({"train": d_train,
                                 "test": d_test,
                                 "thr_err": thr_err,
                                 "lda_err": lda_err,
                                 "lda_w_prior_err": lda_w_prior_err})

# Print
df = pd.DataFrame(results_all_days)
print(df.to_string(index=False))


In [ ]:
# Extract only salami pixels as vector
X_salami = multiIm[salami_an, :].astype(np.float64)

s_salami_w_prior = X_salami @ w + c_w_prior_d

cls_lda = np.zeros(multiIm.shape[:2], dtype=np.uint8)
cls_lda[salami_an] = 1
cls_lda[salami_an] = np.where(s_salami >= 0, 2, 1)
cls_lda_w_prior = np.zeros(multiIm.shape[:2], dtype=np.uint8)
cls_lda_w_prior[salami_an] = 1
cls_lda_w_prior[salami_an] = np.where(s_salami_w_prior >= 0, 2, 1)

plt.figure(figsize=(7, 7))
plt.imshow(cls_lda_w_prior)
plt.title("Classification of LDA with priors")
plt.axis("off")
plt.show()

n_meat = np.sum(cls_lda_w_prior[salami_an] == 1)
n_fat = np.sum(cls_lda_w_prior[salami_an] == 2)
n_roi = np.sum(salami_an)
n_fat_LDA_prior / n_roi